In [13]:
import boto3
import json
import ast

client = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

In [2]:
# import boto3
# import json

# bedrock_region = "us-west-2"
# client = boto3.client("bedrock-runtime", region_name=bedrock_region)

# def generate_claude_response(prompt):
#     model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

#     response = client.converse_stream(
#         modelId=model_id,
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"text": prompt}
#                 ],
#             }
#         ],
#         inferenceConfig={
#             "maxTokens": 10000,
#             "temperature": 0.7,
#             "topP": 1.0,
#             "stopSequences": []
#         }
#     )

#     full_response = ""

#     # Process the streamed response correctly
#     for event in response["stream"]:
#         # print("event:", event)  # Debugging: Print raw event

#         # Extract text from 'contentBlockDelta' events
#         if "contentBlockDelta" in event:
#             full_response += event["contentBlockDelta"]["delta"].get("text", "")

#     return full_response 

In [14]:
def process_stream_response(response):
    try:
        result = ""
        input_tokens, output_tokens = 0, 0
        for event in response["body"]:
            chunk = json.loads(event["chunk"]["bytes"])

            if "delta" in chunk and "text" in chunk["delta"]:
                result += chunk["delta"]["text"]

            if chunk.get("type") == "message_stop" and "amazon-bedrock-invocationMetrics" in chunk:
                metrics = chunk["amazon-bedrock-invocationMetrics"]
                input_tokens = metrics.get("inputTokenCount", 0)
                output_tokens = metrics.get("outputTokenCount", 0)

        return result, input_tokens, output_tokens
    except Exception as e:
        raise RuntimeError(f"Error processing Claude stream response: {str(e)}")


def generate_stream_response(prompt: str, max_tokens: int = 10000, temperature: float = 0.7):
    try:
        payload = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "temperature": temperature,
            "messages": [{"role": "user", "content": prompt}],
        }

        response = client.invoke_model_with_response_stream(
            body=json.dumps(payload),
            modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            accept="application/json",
            contentType="application/json"
        )
        return process_stream_response(response)
    except Exception as e:
        raise RuntimeError(f"Error generating Claude stream response: {str(e)}")

In [19]:
def generate_response(prompt: str, max_tokens: int = 10000, temperature: float = 1, top_k: int = 250,
                      top_p: float = 0.999):
    try:
        payload = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "top_k": top_k,
            "temperature": temperature,
            "top_p": top_p,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}],
                }
            ],
        }

        response = client.invoke_model(
            body=json.dumps(payload),
            modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            accept="application/json",
            contentType="application/json"
        )

        body = json.loads(response["body"].read())
        text = body["content"][0]["text"]
        return text, 0, 0
    except Exception as e:
        print(traceback.format_exc())
        raise RuntimeError(f"Error generating Claude response: {str(e)}")

In [20]:
# def check_for_table(result):
#     try:
#         # Convert string representation to a Python list
#         result_list = ast.literal_eval(result)

#         # Ensure the result is a list with at least 2 entries
#         if isinstance(result_list, list) and len(result_list) > 1:
#             for row in result_list:
#                 if isinstance(row, dict):  # Ensure it's a dictionary
#                     has_non_float = any(not isinstance(value, float) for value in row.values())  # Accepts int & str
#                     has_number = any(isinstance(value, (int, float)) for value in row.values())

#                     if has_non_float and has_number:
#                         return True  # Use table format

#         return False  # Otherwise, don't use table format

#     except (ValueError, SyntaxError):
#         return False  # If parsing fails, assume it's not table-compatible

In [21]:
def check_for_table(result):
    """Determine if result should be displayed as a table."""
    try:
        # result_list = self.result.get("table")
        result_list = result

        if isinstance(result_list, list) and len(result_list) > 1:
            return all(isinstance(row, dict) for row in result_list)
        return False
    except (ValueError, SyntaxError):
        return False

In [22]:
def get_prompt(file):
    try:
        with open(file, "r") as f:
            return f.read()
    except Exception as e:
        raise f"Error reading file: {str(e)}"

In [23]:
def generate_html_response(result, question):
    result = result
    question = question

    is_table_possible = check_for_table(result)
    # Determine the correct prompt
    print(is_table_possible)
    if is_table_possible:
        html_prompt = get_prompt('neptune_table_html_prompt_AG.txt')
        # html_prompt = get_prompt('test_print_res_prompt.txt')
    else:
        html_prompt = get_prompt('neptune_list_html_prompt.txt')

    # Format the response
    result_str = str(result)
    html_prompt = html_prompt.replace('$$RESULT$$', result_str).replace('$$QUESTION$$', question)
    answer = generate_response(html_prompt)
    res_stream = generate_stream_response(html_prompt)

    return answer, res_stream

In [24]:
result = """[{'Neighborhood': 'Cluster 1', 'Shift': 'DAY', 'Number of Offenses': 56}, {'Neighborhood': 'Cluster 1', 'Shift': 'EVENING', 'Number of Offenses': 69}, {'Neighborhood': 'Cluster 1', 'Shift': 'MIDNIGHT', 'Number of Offenses': 34}, {'Neighborhood': 'Cluster 10', 'Shift': 'DAY', 'Number of Offenses': 16}, {'Neighborhood': 'Cluster 10', 'Shift': 'EVENING', 'Number of Offenses': 27}, {'Neighborhood': 'Cluster 10', 'Shift': 'MIDNIGHT', 'Number of Offenses': 5}, {'Neighborhood': 'Cluster 11', 'Shift': 'DAY', 'Number of Offenses': 34}, {'Neighborhood': 'Cluster 11', 'Shift': 'EVENING', 'Number of Offenses': 45}, {'Neighborhood': 'Cluster 11', 'Shift': 'MIDNIGHT', 'Number of Offenses': 11}, {'Neighborhood': 'Cluster 12', 'Shift': 'DAY', 'Number of Offenses': 28}, {'Neighborhood': 'Cluster 12', 'Shift': 'EVENING', 'Number of Offenses': 39}, {'Neighborhood': 'Cluster 12', 'Shift': 'MIDNIGHT', 'Number of Offenses': 5}, {'Neighborhood': 'Cluster 13', 'Shift': 'DAY', 'Number of Offenses': 17}, {'Neighborhood': 'Cluster 13', 'Shift': 'EVENING', 'Number of Offenses': 15}, {'Neighborhood': 'Cluster 13', 'Shift': 'MIDNIGHT', 'Number of Offenses': 2}, {'Neighborhood': 'Cluster 14', 'Shift': 'DAY', 'Number of Offenses': 51}, {'Neighborhood': 'Cluster 14', 'Shift': 'EVENING', 'Number of Offenses': 41}, {'Neighborhood': 'Cluster 14', 'Shift': 'MIDNIGHT', 'Number of Offenses': 12}, {'Neighborhood': 'Cluster 15', 'Shift': 'DAY', 'Number of Offenses': 37}, {'Neighborhood': 'Cluster 15', 'Shift': 'EVENING', 'Number of Offenses': 48}, {'Neighborhood': 'Cluster 15', 'Shift': 'MIDNIGHT', 'Number of Offenses': 8}, {'Neighborhood': 'Cluster 16', 'Shift': 'DAY', 'Number of Offenses': 20}, {'Neighborhood': 'Cluster 16', 'Shift': 'EVENING', 'Number of Offenses': 14}, {'Neighborhood': 'Cluster 16', 'Shift': 'MIDNIGHT', 'Number of Offenses': 6}, {'Neighborhood': 'Cluster 17', 'Shift': 'DAY', 'Number of Offenses': 105}, {'Neighborhood': 'Cluster 17', 'Shift': 'EVENING', 'Number of Offenses': 111}, {'Neighborhood': 'Cluster 17', 'Shift': 'MIDNIGHT', 'Number of Offenses': 46}, {'Neighborhood': 'Cluster 18', 'Shift': 'DAY', 'Number of Offenses': 158}, {'Neighborhood': 'Cluster 18', 'Shift': 'EVENING', 'Number of Offenses': 121}, {'Neighborhood': 'Cluster 18', 'Shift': 'MIDNIGHT', 'Number of Offenses': 73}, {'Neighborhood': 'Cluster 19', 'Shift': 'DAY', 'Number of Offenses': 38}, {'Neighborhood': 'Cluster 19', 'Shift': 'EVENING', 'Number of Offenses': 49}, {'Neighborhood': 'Cluster 19', 'Shift': 'MIDNIGHT', 'Number of Offenses': 30}, {'Neighborhood': 'Cluster 2', 'Shift': 'DAY', 'Number of Offenses': 190}, {'Neighborhood': 'Cluster 2', 'Shift': 'EVENING', 'Number of Offenses': 257}, {'Neighborhood': 'Cluster 2', 'Shift': 'MIDNIGHT', 'Number of Offenses': 88}, {'Neighborhood': 'Cluster 20', 'Shift': 'DAY', 'Number of Offenses': 41}, {'Neighborhood': 'Cluster 20', 'Shift': 'EVENING', 'Number of Offenses': 36}, {'Neighborhood': 'Cluster 20', 'Shift': 'MIDNIGHT', 'Number of Offenses': 16}, {'Neighborhood': 'Cluster 21', 'Shift': 'DAY', 'Number of Offenses': 119}, {'Neighborhood': 'Cluster 21', 'Shift': 'EVENING', 'Number of Offenses': 122}, {'Neighborhood': 'Cluster 21', 'Shift': 'MIDNIGHT', 'Number of Offenses': 67}, {'Neighborhood': 'Cluster 22', 'Shift': 'DAY', 'Number of Offenses': 92}, {'Neighborhood': 'Cluster 22', 'Shift': 'EVENING', 'Number of Offenses': 77}, {'Neighborhood': 'Cluster 22', 'Shift': 'MIDNIGHT', 'Number of Offenses': 67}, {'Neighborhood': 'Cluster 23', 'Shift': 'DAY', 'Number of Offenses': 179}, {'Neighborhood': 'Cluster 23', 'Shift': 'EVENING', 'Number of Offenses': 171}, {'Neighborhood': 'Cluster 23', 'Shift': 'MIDNIGHT', 'Number of Offenses': 93}, {'Neighborhood': 'Cluster 24', 'Shift': 'DAY', 'Number of Offenses': 54}, {'Neighborhood': 'Cluster 24', 'Shift': 'EVENING', 'Number of Offenses': 39}, {'Neighborhood': 'Cluster 24', 'Shift': 'MIDNIGHT', 'Number of Offenses': 32}, {'Neighborhood': 'Cluster 25', 'Shift': 'DAY', 'Number of Offenses': 197}, {'Neighborhood': 'Cluster 25', 'Shift': 'EVENING', 'Number of Offenses': 182}, {'Neighborhood': 'Cluster 25', 'Shift': 'MIDNIGHT', 'Number of Offenses': 63}, {'Neighborhood': 'Cluster 26', 'Shift': 'DAY', 'Number of Offenses': 119}, {'Neighborhood': 'Cluster 26', 'Shift': 'EVENING', 'Number of Offenses': 100}, {'Neighborhood': 'Cluster 26', 'Shift': 'MIDNIGHT', 'Number of Offenses': 31}, {'Neighborhood': 'Cluster 27', 'Shift': 'DAY', 'Number of Offenses': 67}, {'Neighborhood': 'Cluster 27', 'Shift': 'EVENING', 'Number of Offenses': 54}, {'Neighborhood': 'Cluster 27', 'Shift': 'MIDNIGHT', 'Number of Offenses': 27}, {'Neighborhood': 'Cluster 28', 'Shift': 'DAY', 'Number of Offenses': 20}, {'Neighborhood': 'Cluster 28', 'Shift': 'EVENING', 'Number of Offenses': 17}, {'Neighborhood': 'Cluster 28', 'Shift': 'MIDNIGHT', 'Number of Offenses': 11}, {'Neighborhood': 'Cluster 29', 'Shift': 'DAY', 'Number of Offenses': 6}, {'Neighborhood': 'Cluster 29', 'Shift': 'EVENING', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 29', 'Shift': 'MIDNIGHT', 'Number of Offenses': 2}, {'Neighborhood': 'Cluster 3', 'Shift': 'DAY', 'Number of Offenses': 111}, {'Neighborhood': 'Cluster 3', 'Shift': 'EVENING', 'Number of Offenses': 147}, {'Neighborhood': 'Cluster 3', 'Shift': 'MIDNIGHT', 'Number of Offenses': 115}, {'Neighborhood': 'Cluster 30', 'Shift': 'DAY', 'Number of Offenses': 36}, {'Neighborhood': 'Cluster 30', 'Shift': 'EVENING', 'Number of Offenses': 33}, {'Neighborhood': 'Cluster 30', 'Shift': 'MIDNIGHT', 'Number of Offenses': 9}, {'Neighborhood': 'Cluster 31', 'Shift': 'DAY', 'Number of Offenses': 69}, {'Neighborhood': 'Cluster 31', 'Shift': 'EVENING', 'Number of Offenses': 63}, {'Neighborhood': 'Cluster 31', 'Shift': 'MIDNIGHT', 'Number of Offenses': 36}, {'Neighborhood': 'Cluster 32', 'Shift': 'DAY', 'Number of Offenses': 68}, {'Neighborhood': 'Cluster 32', 'Shift': 'EVENING', 'Number of Offenses': 67}, {'Neighborhood': 'Cluster 32', 'Shift': 'MIDNIGHT', 'Number of Offenses': 27}, {'Neighborhood': 'Cluster 33', 'Shift': 'DAY', 'Number of Offenses': 77}, {'Neighborhood': 'Cluster 33', 'Shift': 'EVENING', 'Number of Offenses': 73}, {'Neighborhood': 'Cluster 33', 'Shift': 'MIDNIGHT', 'Number of Offenses': 35}, {'Neighborhood': 'Cluster 34', 'Shift': 'DAY', 'Number of Offenses': 85}, {'Neighborhood': 'Cluster 34', 'Shift': 'EVENING', 'Number of Offenses': 71}, {'Neighborhood': 'Cluster 34', 'Shift': 'MIDNIGHT', 'Number of Offenses': 50}, {'Neighborhood': 'Cluster 35', 'Shift': 'DAY', 'Number of Offenses': 58}, {'Neighborhood': 'Cluster 35', 'Shift': 'EVENING', 'Number of Offenses': 52}, {'Neighborhood': 'Cluster 35', 'Shift': 'MIDNIGHT', 'Number of Offenses': 14}, {'Neighborhood': 'Cluster 36', 'Shift': 'DAY', 'Number of Offenses': 28}, {'Neighborhood': 'Cluster 36', 'Shift': 'EVENING', 'Number of Offenses': 22}, {'Neighborhood': 'Cluster 36', 'Shift': 'MIDNIGHT', 'Number of Offenses': 22}, {'Neighborhood': 'Cluster 37', 'Shift': 'DAY', 'Number of Offenses': 22}, {'Neighborhood': 'Cluster 37', 'Shift': 'EVENING', 'Number of Offenses': 24}, {'Neighborhood': 'Cluster 37', 'Shift': 'MIDNIGHT', 'Number of Offenses': 16}, {'Neighborhood': 'Cluster 38', 'Shift': 'DAY', 'Number of Offenses': 24}, {'Neighborhood': 'Cluster 38', 'Shift': 'EVENING', 'Number of Offenses': 37}, {'Neighborhood': 'Cluster 38', 'Shift': 'MIDNIGHT', 'Number of Offenses': 26}, {'Neighborhood': 'Cluster 39', 'Shift': 'DAY', 'Number of Offenses': 96}, {'Neighborhood': 'Cluster 39', 'Shift': 'EVENING', 'Number of Offenses': 108}, {'Neighborhood': 'Cluster 39', 'Shift': 'MIDNIGHT', 'Number of Offenses': 74}, {'Neighborhood': 'Cluster 4', 'Shift': 'DAY', 'Number of Offenses': 50}, {'Neighborhood': 'Cluster 4', 'Shift': 'EVENING', 'Number of Offenses': 71}, {'Neighborhood': 'Cluster 4', 'Shift': 'MIDNIGHT', 'Number of Offenses': 16}, {'Neighborhood': 'Cluster 40', 'Shift': 'DAY', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 40', 'Shift': 'EVENING', 'Number of Offenses': 4}, {'Neighborhood': 'Cluster 40', 'Shift': 'MIDNIGHT', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 41', 'Shift': 'DAY', 'Number of Offenses': 1}, {'Neighborhood': 'Cluster 43', 'Shift': 'DAY', 'Number of Offenses': 6}, {'Neighborhood': 'Cluster 43', 'Shift': 'EVENING', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 43', 'Shift': 'MIDNIGHT', 'Number of Offenses': 4}, {'Neighborhood': 'Cluster 44', 'Shift': 'DAY', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 44', 'Shift': 'EVENING', 'Number of Offenses': 2}, {'Neighborhood': 'Cluster 44', 'Shift': 'MIDNIGHT', 'Number of Offenses': 1}, {'Neighborhood': 'Cluster 45', 'Shift': 'DAY', 'Number of Offenses': 3}, {'Neighborhood': 'Cluster 45', 'Shift': 'EVENING', 'Number of Offenses': 4}, {'Neighborhood': 'Cluster 45', 'Shift': 'MIDNIGHT', 'Number of Offenses': 1}, {'Neighborhood': 'Cluster 46', 'Shift': 'EVENING', 'Number of Offenses': 1}, {'Neighborhood': 'Cluster 5', 'Shift': 'DAY', 'Number of Offenses': 41}, {'Neighborhood': 'Cluster 5', 'Shift': 'EVENING', 'Number of Offenses': 37}, {'Neighborhood': 'Cluster 5', 'Shift': 'MIDNIGHT', 'Number of Offenses': 18}, {'Neighborhood': 'Cluster 6', 'Shift': 'DAY', 'Number of Offenses': 106}, {'Neighborhood': 'Cluster 6', 'Shift': 'EVENING', 'Number of Offenses': 124}, {'Neighborhood': 'Cluster 6', 'Shift': 'MIDNIGHT', 'Number of Offenses': 81}, {'Neighborhood': 'Cluster 7', 'Shift': 'DAY', 'Number of Offenses': 83}, {'Neighborhood': 'Cluster 7', 'Shift': 'EVENING', 'Number of Offenses': 95}, {'Neighborhood': 'Cluster 7', 'Shift': 'MIDNIGHT', 'Number of Offenses': 64}, {'Neighborhood': 'Cluster 8', 'Shift': 'DAY', 'Number of Offenses': 140}, {'Neighborhood': 'Cluster 8', 'Shift': 'EVENING', 'Number of Offenses': 163}, {'Neighborhood': 'Cluster 8', 'Shift': 'MIDNIGHT', 'Number of Offenses': 52}, {'Neighborhood': 'Cluster 9', 'Shift': 'DAY', 'Number of Offenses': 53}, {'Neighborhood': 'Cluster 9', 'Shift': 'EVENING', 'Number of Offenses': 58}, {'Neighborhood': 'Cluster 9', 'Shift': 'MIDNIGHT', 'Number of Offenses': 24}, {'Neighborhood': None, 'Shift': 'EVENING', 'Number of Offenses': 1}, {'Neighborhood': None, 'Shift': 'MIDNIGHT', 'Number of Offenses': 1}, {'Neighborhood': None, 'Shift': 'DAY', 'Number of Offenses': 1}] """
question = "How does the frequency of offenses vary by shift across different neighborhoods?"

In [25]:
# result = [{'Task ID': '1020', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '120'}, {'Task ID': '1021', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '120'}, {'Task ID': '1023', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1024', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1031', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1037', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1038', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1039', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1025', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '121'}, {'Task ID': '1017', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '122'}, {'Task ID': '1018', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '122'}, {'Task ID': '1041', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1042', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1043', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1048', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1054', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1055', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1056', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 0.0, 'Order ID': '123'}, {'Task ID': '1036', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 535.0, 'Order ID': '121'}, {'Task ID': '1053', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 535.0, 'Order ID': '123'}, {'Task ID': '1029', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 750.0, 'Order ID': '121'}, {'Task ID': '1046', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 750.0, 'Order ID': '123'}, {'Task ID': '1027', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 781.0, 'Order ID': '121'}, {'Task ID': '1045', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 781.0, 'Order ID': '123'}, {'Task ID': '1030', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 794.0, 'Order ID': '121'}, {'Task ID': '1047', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 794.0, 'Order ID': '123'}, {'Task ID': '1028', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1013.0, 'Order ID': '121'}, {'Task ID': '1033', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1058.0, 'Order ID': '121'}, {'Task ID': '1050', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1058.0, 'Order ID': '123'}, {'Task ID': '1026', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1545.0, 'Order ID': '121'}, {'Task ID': '1044', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1545.0, 'Order ID': '123'}, {'Task ID': '1035', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1693.0, 'Order ID': '121'}, {'Task ID': '1052', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1693.0, 'Order ID': '123'}, {'Task ID': '1032', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1840.0, 'Order ID': '121'}, {'Task ID': '1049', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 1840.0, 'Order ID': '123'}, {'Task ID': '1019', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 2012.0, 'Order ID': '120'}, {'Task ID': '1022', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 2012.0, 'Order ID': '121'}, {'Task ID': '1016', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 2012.0, 'Order ID': '122'}, {'Task ID': '1040', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 2012.0, 'Order ID': '123'}, {'Task ID': '1034', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 4245.0, 'Order ID': '121'}, {'Task ID': '1051', 'Task Creation Date': '2023-07-19T00:00:00Z', 'Task Labor Price': 4245.0, 'Order ID': '123'}]
# question = """ For all "FL_Order" for "Part_ID" \'255\' list all "FL_Tasks" by "FL_Task_Creation_Date" and "FL_Task_Labor_Price" """

In [26]:
# result = """  [{'Year': 2018, 'Total Revenue': 24564350.0}, {'Year': 2019, 'Total Revenue': 22275220.0}, {'Year': 2020, 'Total Revenue': 20999940.0}, {'Year': 2021, 'Total Revenue': 15893080.0}, {'Year': 2022, 'Total Revenue': 17132090.0}, {'Year': 2023, 'Total Revenue': 20705560.0}, {'Year': 2024, 'Total Revenue': 10936850.0}] """
# question = "give me total revenue by year"

In [ ]:
import time
# result = """[{\'Account Number\': \'01-000-1338\', \'Description\': \'WIP Parts/Repairs\', \'Credit Value\': 120440.97, \'Debit Value\': 65588.6}]"""
# question = "List total credit value and total debit value by account number and description"

start_time = time.time()
res, stream = generate_html_response(result, question)
end_time = time.time()
time_taken = end_time - start_time
print("time_taken:", time_taken)
print("result:", res)
print(f"\n stream result:- {stream}")

False
